## 句子关系推断（Entailment）
- 一次性提供2个句子给模型推断这两句话有什么含义

目的：判断2句话是否有相连的关系

In [2]:
import torch
import random

from transformers import AutoTokenizer

#加载tokenizer
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')

tokenizer

BertTokenizerFast(name_or_path='google-bert/bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

## 加载数据集并处理

In [3]:
from datasets import load_dataset

#加载数据集
dataset = load_dataset(path='lansinuote/ChnSentiCorp')

#过滤句子长度
f = lambda x: len(x['text']) >= 40
dataset = dataset.filter(f)

#移除多余的字段
dataset = dataset.remove_columns(['label'])

dataset, dataset['train'][0]

Filter: 100%|██████████| 1200/1200 [00:00<00:00, 140442.12 examples/s]


(DatasetDict({
     train: Dataset({
         features: ['text'],
         num_rows: 8130
     })
     validation: Dataset({
         features: ['text'],
         num_rows: 1032
     })
     test: Dataset({
         features: ['text'],
         num_rows: 1011
     })
 }),
 {'text': '选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般'})

## 定义loader

In [4]:
#定义数据集遍历工具
def collate_fn(data):
    b = len(data)
    text = [i['text'] for i in data]

    #生成前后两段话分别的索引
    s1 = list(range(b))
    s2 = list(range(b))
    random.shuffle(s2)

    #根据索引生成label,表明两句话是否是前后相连的关系
    label = [s1[i] == s2[i] for i in range(b)]

    #取出具体的文字  在第20个字符处切分
    s1 = [text[i][0:20] for i in s1]
    s2 = [text[i][20:40] for i in s2]

    #句子对编码
    data = tokenizer(s1,
                     s2,
                     padding=True,
                     truncation=True,
                     max_length=50,
                     return_tensors='pt')

    #设置label
    data['label'] = torch.LongTensor(label)

    return data


loader = torch.utils.data.DataLoader(dataset['train'],
                                     batch_size=4,
                                     shuffle=True,
                                     drop_last=True,
                                     collate_fn=collate_fn)

data = next(iter(loader))

for k, v in data.items():
    print(k, v.shape)

len(loader)

input_ids torch.Size([4, 43])
token_type_ids torch.Size([4, 43])
attention_mask torch.Size([4, 43])
label torch.Size([4])


2032

## 数据样例

In [5]:
#查看数据样例
for input_ids, label in zip(data['input_ids'], data['label']):
    print(tokenizer.decode(input_ids))
    print(label)
    print('================')

[CLS] 酒 店 服 务 不 好 ， 服 务 员 态 度 生 硬 ， 特 别 是 酒 店 [SEP] 地 毯 、 卫 生 间 和 床 铺 都 很 脏 ， 我 的 孩 子 睡 了 一 [SEP]
tensor(0)
[CLS] 我 的 孩 子 4 周 岁 又 5 个 月 了 ， 我 买 了 《 迷 宫 》 [SEP] 层 很 高 。 房 间 从 来 没 有 安 排 到 过 正 面 湖 景 的 ， [SEP]
tensor(0)
[CLS] 从 门 面 来 看 ， 该 酒 店 就 显 得 不 够 气 派 ， 虽 然 楼 [SEP] 物 品 管 理 方 面 ， 物 品 的 清 点 不 到 位 ， 我 明 明 没 [SEP]
tensor(0)
[CLS] 这 个 酒 店 很 烂 。 大 堂 嘈 杂 ， 房 间 设 备 陈 旧 ， [SEP] 4 岁 ， 5 岁 ， 6 岁 的 ， 因 为 她 喜 欢 走 迷 宫 ， 而 [SEP] [PAD]
tensor(0)


## 定义下游任务模型（Downstream Tasks by Pytorch)

In [6]:
#定义模型
class Model(torch.nn.Module):

    def __init__(self):
        super().__init__()

        #加载预训练模型
        from transformers import AutoModel
        self.pretrained = AutoModel.from_pretrained(
            'google-bert/bert-base-chinese')

        self.fc = torch.nn.Linear(in_features=768, out_features=2)

    def forward(self, input_ids, attention_mask, token_type_ids, label=None):
        #使用预训练模型抽取数据特征
        with torch.no_grad():
            last_hidden_state = self.pretrained(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids).last_hidden_state

        #只取第0个词的特征做分类,这和bert模型的训练方式有关,此处不展开
        last_hidden_state = last_hidden_state[:, 0]

        #对抽取的特征只取第一个字的结果做分类即可
        out = self.fc(last_hidden_state).softmax(dim=1)

        #计算loss
        loss = None
        if label is not None:
            loss = torch.nn.functional.cross_entropy(out, label)

        return loss, out


model = Model()

model(**data)

(tensor(0.6777, grad_fn=<NllLossBackward0>),
 tensor([[0.4044, 0.5956],
         [0.6440, 0.3560],
         [0.4996, 0.5004],
         [0.5289, 0.4711]], grad_fn=<SoftmaxBackward0>))

## 执行训练

In [7]:
#执行训练
def train():
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    for i, data in enumerate(loader):
        loss, out = model(**data)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i % 10 == 0:
            out = out.argmax(dim=1)
            acc = (out == data.label).sum().item() / len(data.label)
            print(i, len(loader), loss.item(), acc)

        if i == 300:
            break


train()

0 2032 0.7160650491714478 0.5
10 2032 0.614642858505249 0.75
20 2032 0.6163564324378967 0.75
30 2032 0.496163547039032 1.0
40 2032 0.476115345954895 1.0
50 2032 0.7115070819854736 0.5
60 2032 0.5616160035133362 0.75
70 2032 0.5783414244651794 0.5
80 2032 0.42220208048820496 1.0
90 2032 0.38469523191452026 1.0
100 2032 0.4045517146587372 1.0
110 2032 0.3656919002532959 1.0
120 2032 0.43291038274765015 1.0
130 2032 0.4087420701980591 1.0
140 2032 0.45405665040016174 0.75
150 2032 0.5398662090301514 0.75
160 2032 0.44411206245422363 0.75
170 2032 0.533974289894104 0.5
180 2032 0.46027106046676636 1.0
190 2032 0.3544018268585205 1.0
200 2032 0.391356885433197 1.0
210 2032 0.4554879069328308 1.0
220 2032 0.3896465003490448 1.0
230 2032 0.46633249521255493 1.0
240 2032 0.47276103496551514 1.0
250 2032 0.6206538677215576 0.5
260 2032 0.3760732412338257 1.0
270 2032 0.4522066116333008 0.75
280 2032 0.449824720621109 0.75
290 2032 0.6924847960472107 0.5
300 2032 0.36266255378723145 1.0


## 执行测试

In [8]:
#执行测试
def test():
    loader_test = torch.utils.data.DataLoader(dataset['test'],
                                              batch_size=4,
                                              shuffle=True,
                                              drop_last=True,
                                              collate_fn=collate_fn)

    correct = 0
    total = 0
    for i, data in enumerate(loader_test):
        with torch.no_grad():
            _, out = model(**data)

        out = out.argmax(dim=1)
        correct += (out == data.label).sum().item()
        total += len(data.label)

        print(i, len(loader_test), correct / total)

        if i == 5:
            break

    return correct / total


test()

0 252 1.0
1 252 1.0
2 252 1.0
3 252 0.9375
4 252 0.95
5 252 0.9166666666666666


0.9166666666666666